In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fastai.tabular.all import *


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')


In [ ]:
df_train = pd.read_csv(input_path/'train.csv', index_col = 'id')
df_test = pd.read_csv(input_path/'test.csv', index_col = 'id')
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')

In [ ]:
df_train['target'].describe()

In [ ]:
columns_outlier_low = ['target','cont10','cont9','cont7','cont1']
columns_outlier_up = ['cont10']

In [ ]:
n999 = [ np.percentile(df_train[i],99.9) for i in df_train[columns_outlier_up]]
n001 = [ np.percentile(df_train[i],0.1) for i in df_train[columns_outlier_low]]

In [ ]:
for c in columns_outlier_up:
   
    df_train.drop(df_train[df_train[c] > n999[0]].index, inplace = True)

In [ ]:
for nn, c in zip(n001,columns_outlier_low):
    df_train.drop(df_train[df_train[c] < nn].index, inplace = True)
     

In [ ]:
cont_nn = df_train.columns
cont_nn = cont_nn.drop('target').tolist()

In [ ]:
dls = TabularDataLoaders.from_df(df_train, cat_names =[], cont_names= cont_nn,y_names ='target', procs = [Normalize], bs = 512)

In [ ]:
learn = tabular_learner(dls, metrics = rmse, y_range=(0,10.5), layers=[500,250],
                        n_out=1, loss_func=F.mse_loss, wd = 0.1)

In [ ]:
 learn.lr_find()

In [ ]:
learn.fit_one_cycle(n_epoch=30,lr_max=0.003)

In [ ]:
learn.recorder.plot_loss(skip_start=300)

In [ ]:
learn.export()

In [ ]:
dl = learn.dls.test_dl(df_test)

In [ ]:
preds, _ =learn.get_preds(dl=dl)
preds = preds.squeeze().numpy()

In [ ]:
submission['target'] = preds

In [ ]:
submission.to_csv('fastai.csv')